In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("http://localhost:3030/InvestigacionesUdenar/sparql")

### Cargamos Vocabulario y Adicionales

In [3]:
import difflib

#Lectura de lemas 
import json
with open('./Vocabulario/diccionario-lemas.json', 'r') as f:
    diccionario_lemas = json.loads(f.read())

lemas = list(diccionario_lemas.keys())

### Cargamos Modelo W2V

In [4]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import multiprocessing
from time import time

modelo = Word2Vec.load("./Modelos/modelo10FINAL.model")


### Lectura de Palabras

In [89]:
words = ['gestor', 'conocimiento']
#Modelo W2v
similar_words = modelo.wv.most_similar(words, topn=20)

words.extend([word[0] for word in similar_words])

#Palabras en plural o muy similares
more_words=[]
for word in words:
    additional = difflib.get_close_matches(word, diccionario_lemas.keys(), n=5, cutoff=0.90)
    more_words.extend(additional[1:])
    
#Añadir palabras finales
words.extend(more_words)
words

['gestor',
 'conocimiento',
 'debilmente',
 'umayux',
 'sgbd',
 'sparql',
 'rdf',
 'postgresql',
 'ontologias',
 'fundamentandose',
 'solicitada',
 'maskana',
 'motor',
 'transformarla',
 'facilite',
 'adquieran',
 'buscador',
 'digitalizados',
 'flock',
 'robusto',
 'publicarlo',
 'divulgarlo',
 'reconocimiento',
 'solicitado',
 'transformarlo',
 'transformar',
 'buscado',
 'flocks']

### Cargamos palabras a SPARQL 

In [94]:
query_prefix = """
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?palabras po:descripcion_palabra ?descripcion.
        ?palabras ^po:pi_tiene_palabra ?proyecto_investigacion.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        FILTER ( """

query_words=""""""
query_words_start = """REGEX(str(?descripcion), '\\\\b"""
query_words_final = """\\\\b','i') || """
query_words_final_string = """\\\\b','i')"""

for i, word in enumerate(words):
    if i < len(words)-1:
        query_words += query_words_start+word+query_words_final
    else:
        query_words += query_words_start+word+query_words_final_string
        

query_close= """)
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
"""

query = query_prefix+query_words+query_close
#print(query)

### Realizamos la consulta

In [95]:
sparql.setQuery(query)

#order by DESC(?c)

sparql.setReturnFormat(JSON)

results = sparql.query().convert()
results = results['results']['bindings']

### Mostramos Resultados

In [96]:
for i, result in enumerate(results):
    print(f'({i+1}) -> ' + result['Titulo']['value'])

(1) -> UMAYUX: UN MODELO DE SOFTWARE DE GESTION DE CONOCIMIENTO SOPORTADO EN UNA ONTOLOGIA DINAMICA DEBILMENTE ACOPLADO CON UN GESTOR DE BASE DE DATOS PARA LA UNIVERSIDAD DE NARIÑO
(2) -> MOTOR DE BÚSQUEDA INTELIGENTE DE INFORMES DE INVESTIGACIÓN BASADO EN RECURSOS SEMÁNTICOS PARA EL SISTEMA DE INVESTIGACIONES DE LA UNIVERSIDAD DE NARIÑO
(3) -> APROXIMACIÓN AL PENSAMIENTO JURÍDICO NARIÑENSE DESDE LAS FIGURAS DE JOSÉ RAFAEL SAÑUDO Y ERNESTO VELA ANGULO.
(4) -> SITAPP: UNA APLICACIÓN INTELIGENTE PARA DISPOSITIVOS MÓVILES DEL SISTEMA DE RUTAS DE TRANSPORTE URBANO DEL MUNICIPIO DE PASTO
(5) -> "Diagnóstico del impacto de las iniciativas emprendedoras de los egresados del programa de diseño industrial de la Universidad de Nariño"
(6) -> ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE INTELIGENCIA DE NEGOCIOS DE SOFTWARE LIBRE SPAGO BI ORIENTADA AL USO EN LAS PYMES DE LA REGIÓN
(7) -> ANÁLISIS DE LA CONTRIBUCIÓN SOCIOECONÓMICA DE LAS MUJERES TRABAJADORAS EN LAS ZONAS RURALES DEL MUNICIPIO DE 

In [97]:
id_docs = [result['id_pi']['value'] for result in results ]
len(id_docs)

171

In [2]:
qres = g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?proyecto_investigacion po:pi_tiene_palabra ?palabra.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabra po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
""")


In [11]:
qres = g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?palabras po:palabra_describe_pi ?proyecto_investigacion.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabras po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
""")

In [5]:
#Directa
trabajos = list(graph.query("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?proyecto_investigacion po:pi_tiene_palabra ?palabra.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabra po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
"""))

#trabajos
                